# Cars: Grouping and Aggregation

## Imports

In [1]:
import pandas as pd
import numpy as np
from altair import load_dataset

## Dataset

In [2]:
cars = load_dataset('cars')

In [3]:
cars.head()

,Acceleration,Cylinders,Displacement,Horsepower,Miles_per_Gallon,Name,Origin,Weight_in_lbs,Year
0,12.0,8,307.0,130.0,18.0,chevrolet chevelle malibu,USA,3504,1970-01-01
1,11.5,8,350.0,165.0,15.0,buick skylark 320,USA,3693,1970-01-01
2,11.0,8,318.0,150.0,18.0,plymouth satellite,USA,3436,1970-01-01
3,12.0,8,304.0,150.0,16.0,amc rebel sst,USA,3433,1970-01-01
4,10.5,8,302.0,140.0,17.0,ford torino,USA,3449,1970-01-01


In [4]:
len(cars)

406

## Grouped mean

Compute the average of `Acceleration`, `Displacement` and `Horsepower`, grouped by `Origin`:

In [5]:
# YOUR CODE HERE
g1 = cars.groupby("Origin")[["Acceleration", "Displacement", "Horsepower"]].mean()
g1

,Acceleration,Displacement,Horsepower
Origin,,,
Europe,16.821918,109.465753,81.000000
Japan,16.172152,102.708861,79.835443
USA,14.942520,247.935039,119.900000


In [6]:
assert list(g1.columns)==['Acceleration', 'Displacement', 'Horsepower']
assert list(g1.index)==['Europe', 'Japan', 'USA']

Find the maximum `Acceleration` and `Displacement` grouped by `Origin` and `Cylinders`:

In [7]:
# YOUR CODE HERE
g2 = cars.groupby(["Origin", "Cylinders"])[["Acceleration", "Displacement"]].max()
g2

Acceleration  Displacement
Origin Cylinders                            
Europe 4                  24.8         146.0
       5                  20.1         183.0
       6                  19.6         168.0
Japan  3                  13.5          80.0
       4                  21.0         144.0
       6                  15.5         168.0
USA    4                  22.2         156.0
       6                  21.0         262.0
       8                  22.2         455.0

In [8]:
assert list(g2.columns)==['Acceleration', 'Displacement']
assert [list(i) for i in list(g2.index.levels)]==[['Europe', 'Japan', 'USA'], [3, 4, 5, 6, 8]]

## Grouped counts

Compute the number of cars, grouped by `Year` and `Cylinders` and unstack the result:

In [9]:
# YOUR CODE HERE
g3 = cars.groupby(["Year", "Cylinders"])["Cylinders"].count().unstack()
g3

Cylinders,3,4,5,6,8
Year,,,,,
1970-01-01,NaN,8.0,NaN,4.0,23.0
1971-01-01,NaN,14.0,NaN,8.0,7.0
1972-01-01,1.0,14.0,NaN,NaN,13.0
1973-01-01,1.0,11.0,NaN,8.0,20.0
1974-01-01,NaN,15.0,NaN,7.0,5.0
1975-01-01,NaN,12.0,NaN,12.0,6.0
1976-01-01,NaN,15.0,NaN,10.0,9.0
1977-01-01,1.0,14.0,NaN,5.0,8.0
1978-01-01,NaN,17.0,1.0,12.0,6.0


In [10]:
assert list(g3.columns)==[3, 4, 5, 6, 8]
assert list(g3.index)==['1970-01-01', '1971-01-01', '1972-01-01', '1973-01-01',
                        '1974-01-01', '1975-01-01', '1976-01-01', '1977-01-01', 
                        '1978-01-01', '1979-01-01', '1980-01-01', '1982-01-01']

## Multiple aggregations

Compute the min and max `Miles_per_Gallon` grouped by `Origin`:

In [11]:
# YOUR CODE HERE
g4 = cars.groupby("Origin")["Miles_per_Gallon"].aggregate(["min", "max"])
g4

,min,max
Origin,,
Europe,16.2,44.3
Japan,18.0,46.6
USA,9.0,39.0


In [12]:
assert set(g4.columns)==set(['min', 'max'])
assert list(g4.index)==['Europe', 'Japan', 'USA']

## Custom aggregation function

Compute the range of cylinders (`range` = max - min) grouped by `Origin` and name the resulting column `range`:

In [13]:
# YOUR CODE HERE
def ranging(x):
    return x.max() - x.min()
g5 = cars.groupby("Origin")[["Cylinders"]].aggregate([ranging])
g5.columns = g5.columns.droplevel(0)
g5.columns = ["range"]
g5

,range
Origin,
Europe,2
Japan,3
USA,4


In [14]:
assert list(g5.columns)==['range']
assert list(g5.index)==['Europe', 'Japan', 'USA']

## Group filtering

Compute the average acceleration, grouped by `Year` and `Origin`, only including groups with a max number of Cylinders less than 6. Unstack the `Origin` level of the resulting hierarchical row index:

In [15]:
# YOUR CODE HERE
# not sure the ordering of it
def filter_func(x):
    return x.Cylinders.max() < 6
#g6 = cars.groupby(["Year", "Origin"]).mean().filter([filter_func]).[["Acceleration"]]j.unstack(level = "Origin")
filtered = cars.groupby(["Year", "Origin"]).filter(filter_func)
g6 = filtered.groupby(["Year", "Origin"])[["Acceleration"]].mean().unstack(level = "Origin")
g6

Acceleration           
Origin           Europe      Japan
Year                              
1970-01-01    16.666667  14.750000
1971-01-01    17.400000  16.375000
1972-01-01    18.700000  15.400000
1973-01-01    16.428571        NaN
1974-01-01    15.333333  17.666667
1975-01-01    15.083333  16.000000
1977-01-01    15.000000        NaN
1978-01-01          NaN  16.200000
1979-01-01    18.400000  17.200000
1980-01-01    18.366667        NaN

In [16]:
assert [list(i) for i in list(g6.columns.levels)]==[['Acceleration'], ['Europe', 'Japan']]
assert g6.index.name=='Year'
assert len(g6)==10

## Grouped z-scores

Here is the average `Miles_per_Gallon`, grouped by `Year`:

In [17]:
cars.groupby(['Year'])['Miles_per_Gallon'].mean()

Year
1970-01-01    17.689655
1971-01-01    21.250000
1972-01-01    18.714286
1973-01-01    17.100000
1974-01-01    22.703704
1975-01-01    20.266667
1976-01-01    21.573529
1977-01-01    23.375000
1978-01-01    24.061111
1979-01-01    25.093103
1980-01-01    33.696552
1982-01-01    31.045000
Name: Miles_per_Gallon, dtype: float64

Replace the `Miles_per_Gallon` values by the [z-score](https://en.wikipedia.org/wiki/Standard_score) of that value relative to its group for each `Year`:

In [18]:
def to_zscore(x):
    mean = x.mean()
    sd = x.std()
    return (x - mean) / sd

cars["Miles_per_Gallon"] = cars.groupby(["Year"])["Miles_per_Gallon"].apply(to_zscore)
cars["Miles_per_Gallon"]

0      0.058125
1     -0.503753
2      0.058125
3     -0.316460
4     -0.129168
5     -0.503753
6     -0.691046
7     -0.691046
8     -0.691046
9     -0.503753
10          NaN
11          NaN
12          NaN
13          NaN
14          NaN
15    -0.503753
16    -0.691046
17          NaN
18    -0.503753
19    -0.691046
20     1.181883
21     0.807297
22     0.058125
23     0.620004
24     1.743761
25     1.556468
26     1.369176
27     1.181883
28     1.369176
29     1.556468
         ...   
376   -0.737237
377    0.538575
378   -0.008202
379   -0.372720
380   -0.737237
381   -1.284014
382   -1.466273
383    0.903093
384    1.085352
385   -0.008202
386    1.267611
387    0.903093
388    0.903093
389    0.903093
390    0.538575
391    1.267611
392    0.174057
393    1.267611
394   -1.101755
395    1.267611
396   -0.919496
397   -1.648532
398    0.174057
399    0.903093
400   -0.737237
401   -0.737237
402    2.361164
403    0.174057
404   -0.554978
405   -0.008202
Name: Miles_per_Gallon, 

Show that the average z-scores for `Miles_per_Gallon` grouped by year are all zero:

In [19]:
# YOUR CODE HERE
avg_z = cars.groupby(["Year"])["Miles_per_Gallon"].mean()
avg_z

Year
1970-01-01   -7.656711e-17
1971-01-01    4.758099e-17
1972-01-01   -2.299748e-16
1973-01-01   -3.053113e-16
1974-01-01    2.138207e-16
1975-01-01    2.960595e-16
1976-01-01   -5.224579e-17
1977-01-01    1.189525e-17
1978-01-01   -3.484867e-16
1979-01-01    1.761043e-15
1980-01-01   -3.072255e-16
1982-01-01   -3.114696e-16
Name: Miles_per_Gallon, dtype: float64

Show that the standard deviation of the z-scores for `Miles_per_Gallon` grouped by year are all 
1.0:

In [20]:
# YOUR CODE HERE
std_z = cars.groupby(["Year"])["Miles_per_Gallon"].std()
std_z

Year
1970-01-01    1.0
1971-01-01    1.0
1972-01-01    1.0
1973-01-01    1.0
1974-01-01    1.0
1975-01-01    1.0
1976-01-01    1.0
1977-01-01    1.0
1978-01-01    1.0
1979-01-01    1.0
1980-01-01    1.0
1982-01-01    1.0
Name: Miles_per_Gallon, dtype: float64

## Grouped missing value replacement

## Introduce missing values

Let's introduce some missing values into the `Cylinders` column:

In [21]:
cars['Cylinders'] = np.where(np.random.rand(len(cars)) > 0.8, np.nan, cars['Cylinders'])
cars.Cylinders

0      8.0
1      8.0
2      8.0
3      8.0
4      8.0
5      8.0
6      8.0
7      8.0
8      8.0
9      8.0
10     4.0
11     NaN
12     NaN
13     8.0
14     8.0
15     8.0
16     8.0
17     8.0
18     8.0
19     NaN
20     4.0
21     NaN
22     6.0
23     6.0
24     4.0
25     4.0
26     4.0
27     NaN
28     4.0
29     4.0
      ... 
376    NaN
377    NaN
378    4.0
379    4.0
380    4.0
381    4.0
382    4.0
383    4.0
384    4.0
385    4.0
386    NaN
387    4.0
388    4.0
389    NaN
390    NaN
391    4.0
392    4.0
393    NaN
394    NaN
395    6.0
396    4.0
397    6.0
398    4.0
399    4.0
400    4.0
401    NaN
402    NaN
403    NaN
404    NaN
405    NaN
Name: Cylinders, Length: 406, dtype: float64

Here are the average number of Cylinders, grouped by `Year`:

In [22]:
cars.groupby(['Year'])['Cylinders'].mean()

Year
1970-01-01    6.785714
1971-01-01    5.500000
1972-01-01    5.760000
1973-01-01    6.606061
1974-01-01    5.473684
1975-01-01    5.545455
1976-01-01    5.800000
1977-01-01    5.480000
1978-01-01    5.321429
1979-01-01    5.904762
1980-01-01    4.153846
1982-01-01    4.372093
Name: Cylinders, dtype: float64

Replace the missing values in `Cylinders` by the group average (grouped by `Year`):

In [23]:
# YOUR CODE HERE
def fill_na(x):
    average = x.mean()
    return x.fillna(average)
filled_data = cars.groupby("Year")["Cylinders"].apply(fill_na)
cars["Cylinders"] = filled_data

By replacing missing values with group averages, the grouped averages remain unchanged:

In [24]:
# YOUR CODE HERE
cars.groupby(['Year'])['Cylinders'].mean()

Year
1970-01-01    6.785714
1971-01-01    5.500000
1972-01-01    5.760000
1973-01-01    6.606061
1974-01-01    5.473684
1975-01-01    5.545455
1976-01-01    5.800000
1977-01-01    5.480000
1978-01-01    5.321429
1979-01-01    5.904762
1980-01-01    4.153846
1982-01-01    4.372093
Name: Cylinders, dtype: float64